# Setup

In [1]:
import pandas as pd

TRANSACTION_INDEX = ['RC#', 'Category', 'Vendor#']

def get_tpn_id(i, df):    
    return '-'.join((
        df.at[i, 'RC#'].astype(str),
        df.at[i, 'Category'].astype(str),
        df.at[i, 'Vendor#'].astype(str)
    ))

In [2]:
original = pd.read_csv("./data/original.csv").replace({ 'RC#': { 2: 1 }, 'RC Name': { 'SVC Engagement': 'Chancellor' } })

# Consolidate same-category-and-vendor transactions within an RC

In [3]:
original = original.groupby(['RC#', 'Category', 'Vendor#']).agg({
    # 'RC#': 'first',
    'RC Name': 'first',
    # 'Category': 'first',
    # 'Vendor#': 'first',
    'Vendor Name': 'first',
    'Address 1': 'first',
    'Address 2': 'first',
    'City': 'first',
    'State': 'first',
    'ZIP code': 'first',
    'Amount': 'sum',
}).reset_index()

In [4]:
original = original[[
    'RC#', 'RC Name', 'Category', 'Vendor#', 'Vendor Name', 'Address 1', 'Address 2', 'City', 'State', 'ZIP code', 'Amount'
]]

original = original.sort_values(by=TRANSACTION_INDEX)

for i in original.index:
    original.at[i, 'TPN_ID'] = get_tpn_id(i, original)

In [5]:
original.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5568 entries, 0 to 5567
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   RC#          5568 non-null   int64 
 1   RC Name      5568 non-null   object
 2   Category     5568 non-null   int64 
 3   Vendor#      5568 non-null   int64 
 4   Vendor Name  5568 non-null   object
 5   Address 1    5544 non-null   object
 6   Address 2    1290 non-null   object
 7   City         5469 non-null   object
 8   State        5371 non-null   object
 9   ZIP code     5557 non-null   object
 10  Amount       5568 non-null   int64 
 11  TPN_ID       5568 non-null   object
dtypes: int64(4), object(8)
memory usage: 694.5+ KB


In [6]:
updated = pd.read_csv("./data/updated.csv")
updated = updated.sort_values(by=TRANSACTION_INDEX)

In [7]:
for i in updated.index:
    updated.at[i, 'TPN_ID'] = get_tpn_id(i, updated)

In [8]:
updated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5920 entries, 4352 to 1519
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   RC#          5920 non-null   int64 
 1   RC Name      5920 non-null   object
 2   Category     5920 non-null   int64 
 3   Vendor#      5920 non-null   int64 
 4   Vendor Name  5920 non-null   object
 5   Address 1    5896 non-null   object
 6   Address 2    1359 non-null   object
 7   City         5821 non-null   object
 8   State        5716 non-null   object
 9   ZIP code     5909 non-null   object
 10  Amount       5920 non-null   int64 
 11  TPN_ID       5920 non-null   object
dtypes: int64(4), object(8)
memory usage: 730.3+ KB


# Re-assign transactions from old to new RC if they have the same category/vendor/amount

In [ ]:
merged_transactions = original.merge(updated.drop_duplicates(), on='TPN_ID', how='right', indicator=True)
merged_transactions = merged_transactions[merged_transactions._merge == "right_only"]

for i in merged_transactions.index:
    relevant_transactions = original[
        (original['Category'] == merged_transactions.at[i, 'Category_y']) &
        (original['Vendor#'] == merged_transactions.at[i, 'Vendor#_y']) &
        (original['Amount'] == merged_transactions.at[i, 'Amount_y'])
    ]
    
    if len(relevant_transactions) > 0:
        original.loc[relevant_transactions.head().index[0], "RC#"] = merged_transactions.at[i, 'RC#_y']
        original.loc[relevant_transactions.head().index[0], "RC Name"] = merged_transactions.at[i, 'RC Name_y']

# Re-assign internal ID number due to possible RC changes

In [ ]:
original = original.sort_values(by=TRANSACTION_INDEX)

for i in original.index:
    original.at[i, 'TPN_ID'] = get_tpn_id(i, original)

# Save data

In [ ]:
original.to_csv("./data/original_sorted.csv", index=False)

In [ ]:
updated.to_csv("./data/updated_sorted.csv", index=False)